In [19]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import operator
import tensorflow as tf
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model


import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
import numpy as np 
import pandas as pd 
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
#import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
from keras.models import Model

In [20]:
TRAIN_PATH = r'../input/thesis1/Train'
VAL_PATH = r'../input/thesis1/Test'
BATCH_SIZE=16
r=4
c=4
#CATEGORIES = ['Air_trapping', 'Aortic_elongation','COPD_Signs','Calcified_granuloma','Callus_rib_fracture','Hiatal_hernia','Kyphosis','Laminar_atelectasis','Normal','Pleural_effusion','Scoliosis','Vascular_hilar_enlargement']

In [21]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_batches = train_datagen.flow_from_directory(TRAIN_PATH,
                                                  class_mode='categorical', 
                                                  batch_size=BATCH_SIZE, 
                                                  target_size=(256, 256),
                                                  shuffle=True,
                                                  seed=42
                                                  )

Found 34387 images belonging to 12 classes.


In [22]:
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_batches = val_datagen.flow_from_directory(VAL_PATH,
                                                class_mode='categorical', 
                                          
                                                batch_size=BATCH_SIZE, 
                                                target_size=(256, 256),
                                                shuffle=True,
                                                seed=42
                                                )

Found 8597 images belonging to 12 classes.


In [23]:
from keras.applications.vgg16 import VGG16

In [24]:
from keras.optimizers import Adam

In [25]:
def vgg():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(256,256,3))
    predictions=Dense(12,activation='softmax',trainable=True)(base_model.output)
    for layer in base_model.layers:
        layer.trainable=True
    model=Model(inputs=[base_model.input], outputs=[predictions])
    
    optim = tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-4)
    loss_func = 'mae'
    
    model.compile(optimizer=optim,loss=loss_func,metrics=['accuracy'])
    return model

model=None
model = vgg()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [26]:
#import tensorflow as tf
#from t.keras.optimizers import Adam, RMSprop, SGD
#adam_opt = tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-4)
#sgd_opt = tf.keras.optimizers.SGD(lr=1e-06, momentum=0.9, decay=0.0, nesterov=False)
#rmsp_opt = RMSprop(lr=1e-4, decay=0.9)
# eve_opt = Eve(lr=1e-4, decay=1E-4, beta_1=0.9, beta_2=0.999, beta_3=0.999, small_k=0.1, big_K=10, epsilon=1e-08)

#model.compile(optimizer= adam_opt,
             # loss = 'categorical_crossentropy',
              #metrics=['accuracy'])

In [27]:
#import keras
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('adam_baseline_vgg.h5', monitor='val_accuracy', save_best_only=True, mode='max'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, verbose=1, patience=5, mode='max')]


In [28]:
history  = model.fit(train_batches, 
         steps_per_epoch=train_batches.n//train_batches.batch_size,
         validation_data=val_batches, 
         validation_steps=val_batches.n//val_batches.batch_size, 
         epochs=50, 
         verbose=1,
         callbacks = callbacks)

Epoch 1/50
  88/2149 [>.............................] - ETA: 1:20:41 - loss: 0.0662 - accuracy: 0.7404

KeyboardInterrupt: 

In [29]:
model.save('baseline.h5')

In [33]:
test_generator = ImageDataGenerator()
test_data_generator = test_generator.flow_from_directory(
    '../input/thesis1/Test', # Put your path here
     target_size=(256, 256),
    batch_size=32,
    shuffle=False)
test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

predictions = model.predict_generator(test_data_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

Found 8597 images belonging to 12 classes.


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


KeyboardInterrupt: 

In [ ]:
true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())   

In [ ]:
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    